In [1]:
import numpy as np
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_path = "./data/"

In [4]:
player_stat = pd.read_excel(data_path + "players stat.xlsx")

In [5]:
player_cv = pd.read_excel(data_path + "players cv.xlsx")

In [6]:
player_salary = pd.read_excel(data_path + "players salary.xlsx")

# Season 2015-2016

In [7]:
player_stat_15_16 = player_stat[player_stat['Season'] == '2015-16']

In [8]:
player_salary_15_16 = player_salary[player_salary['SEASON'] == '2015-2016']

In [9]:
player_cv_15_16 = player_cv[(player_cv['From']<=2016) & (player_cv['To']>=2015)]

## Merge Stat and CV

In [10]:
merge_stat_cv = pd.merge(player_stat_15_16, player_cv_15_16, left_on='Player', right_on='Player', how='inner')

In [11]:
merge_stat_cv.shape

(470, 38)

## Merge Stat and CV and Salary

In [12]:
def preprocess_name(name):
    if ',' in name:
        ind = name.find(',')
        name = name[:ind]
    ### We remove the Jr.
    if "Jr." in name:
        name = name.replace("Jr.", "")
    if "Jr" in name:
        name = name.replace("Jr.", "")
    if "III" in name:
        name = name.replace("III", "")
    while '.' in name:
        ind = name.find('.')
        name = name.replace('.','')
    name = name.lower()
    
    return name

In [13]:
player_salary_15_16['Player'] = player_salary_15_16['NAME'].apply(lambda s: preprocess_name(s))

In [14]:
merge_stat_cv['Player'] = merge_stat_cv['Player'].apply(lambda s: preprocess_name(s))

In [15]:
len(set(merge_stat_cv['Player']))

470

In [16]:
final_merge = pd.merge(merge_stat_cv, player_salary_15_16, left_on='Player', right_on='Player', how='inner')

In [17]:
final_merge.shape

(377, 43)

## Final dataframe to work on for 2015-2016

In [18]:
final_merge.head(6)

Player   Season   Age   Tm   Lg   G    GS      MP   PER  \
0  thanasis antetokounmpo  2015-16  23.0  NYK  NBA   2   0.0     6.0  32.9   
1           stephen curry  2015-16  27.0  GSW  NBA  79  79.0  2700.0  31.5   
2            kevin durant  2015-16  27.0  OKC  NBA  72  72.0  2578.0  28.2   
3        boban marjanovic  2015-16  27.0  SAS  NBA  54   4.0   508.0  27.7   
4       russell westbrook  2015-16  27.0  OKC  NBA  80  80.0  2750.0  27.6   
5            lebron james  2015-16  31.0  CLE  NBA  76  76.0  2709.0  27.5   

    3PAr    FTr  ORB%  DRB%  TRB%  AST%  STL%  BLK%  TOV%  USG%   ORtg   DRtg  \
0  0.250  0.000   0.0  17.9   9.1   0.0   0.0   0.0   0.0  30.2  140.0  110.0   
1  0.554  0.250   2.9  13.6   8.6  33.7   3.0   0.4  12.9  32.6  125.0  103.0   
2  0.348  0.361   2.0  21.8  12.4  24.2   1.3   2.5  13.5  30.6  122.0  104.0   
3  0.000  0.655  16.9  26.2  21.7   6.6   1.2   3.5  11.5  22.8  130.0   96.0   
4  0.236  0.397   6.1  18.1  12.4  49.6   2.9   0.6  16.8  31.6  115.0  103.0   
5  0.199  0.347   4.7  18.8  11.8  36.0   2.0   1.5  13.2  31.4  116.0  103.0   

    OWS  DWS    WS  WS/48  OBPM  DBPM   BPM  VORP        Place_of_Birth  \
0   0.0  0.0   0.0  0.291   6.3 -10.5  -4.1   0.0                Greece   
1  13.8  4.1  17.9  0.318  12.4   0.1  12.5   9.8                  Ohio   
2  11.0  3.5  14.5  0.270   7.0   0.9   7.9   6.4  District of Columbia   
3   2.3  1.2   3.4  0.325   2.7   0.9   3.6   0.7                Russia   
4  10.0  4.0  14.0  0.245   7.6   2.4  10.0   8.3            California   
5   9.6  4.0  13.6  0.242   6.9   2.3   9.1   7.6                  Ohio   

              Race  From    To  Pos   Ht   Wt          Birth Date  \
0            Black  2016  2016    F  6-7  205       July 17, 1992   
1  Black and White  2010  2016    G  6-3  185      March 14, 1988   
2            Black  2008  2016    F  6-9  240  September 29, 1988   
3            White  2016  2016    C  7-3  290     August 15, 1988   
4            Black  2009  2016    G  6-3  200   November 12, 1988   
5            Black  2004  2016  F-G  6-8  250   December 30, 1984   

                                 College     SEASON   RK  \
0                                    NaN  2015-2016  416   
1                       Davidson College  2015-2016   55   
2          University of Texas at Austin  2015-2016    7   
3                                    NaN  2015-2016  315   
4  University of California, Los Angeles  2015-2016   16   
5                                    NaN  2015-2016    2   

                         NAME                   TEAM    SALARY  
0  Thanasis Antetokounmpo, SF        New York Knicks     30888  
1           Stephen Curry, PG  Golden State Warriors  11370786  
2            Kevin Durant, SF  Oklahoma City Thunder  20158622  
3         Boban Marjanovic, C      San Antonio Spurs   1200000  
4       Russell Westbrook, PG  Oklahoma City Thunder  16744218  
5            LeBron James, SF    Cleveland Cavaliers  22970500

## Predicting the salary using only numercial values

In [19]:
def convert_height(height_str):
    foot, inches = height_str.split('-')
    height_cm = 30.48 * float(foot) + 2.54* float(inches) 
    return height_cm


def age(ref, date):
    idx = date.find(', ')
    year = int(date[idx+2:])
    return ref - year

In [20]:
final_merge['New age'] = final_merge['Birth Date'].apply(lambda d: age(2015, d))

In [21]:
final_merge.loc[:, 'Ht'] = final_merge['Ht'].apply(lambda h: convert_height(h))

In [22]:
perf_columns = ['G', 'GS', 'MP', 'PER', '3PAr', 
               'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%',
               'STL%', 'BLK%', 'TOV%', 'USG%',
               'ORtg', 'DRtg', 'OWS', 'DWS', 'WS', 
               'WS/48', 'OBPM', 'DBPM', 'BPM','VORP']
perf_sal_df = final_merge.loc[:, perf_columns + ['SALARY']]
perf_sal_df.head()

G    GS      MP   PER   3PAr    FTr  ORB%  DRB%  TRB%  AST%  STL%  BLK%  \
0   2   0.0     6.0  32.9  0.250  0.000   0.0  17.9   9.1   0.0   0.0   0.0   
1  79  79.0  2700.0  31.5  0.554  0.250   2.9  13.6   8.6  33.7   3.0   0.4   
2  72  72.0  2578.0  28.2  0.348  0.361   2.0  21.8  12.4  24.2   1.3   2.5   
3  54   4.0   508.0  27.7  0.000  0.655  16.9  26.2  21.7   6.6   1.2   3.5   
4  80  80.0  2750.0  27.6  0.236  0.397   6.1  18.1  12.4  49.6   2.9   0.6   

   TOV%  USG%   ORtg   DRtg   OWS  DWS    WS  WS/48  OBPM  DBPM   BPM  VORP  \
0   0.0  30.2  140.0  110.0   0.0  0.0   0.0  0.291   6.3 -10.5  -4.1   0.0   
1  12.9  32.6  125.0  103.0  13.8  4.1  17.9  0.318  12.4   0.1  12.5   9.8   
2  13.5  30.6  122.0  104.0  11.0  3.5  14.5  0.270   7.0   0.9   7.9   6.4   
3  11.5  22.8  130.0   96.0   2.3  1.2   3.4  0.325   2.7   0.9   3.6   0.7   
4  16.8  31.6  115.0  103.0  10.0  4.0  14.0  0.245   7.6   2.4  10.0   8.3   

     SALARY  
0     30888  
1  11370786  
2  20158622  
3   1200000  
4  16744218

In [23]:
## Droping the NaN values
perf_sal_df.dropna(inplace=True)

In [24]:
perf_sal_df[perf_columns]

G    GS      MP   PER   3PAr    FTr  ORB%  DRB%  TRB%  AST%  STL%  BLK%  \
0     2   0.0     6.0  32.9  0.250  0.000   0.0  17.9   9.1   0.0   0.0   0.0   
1    79  79.0  2700.0  31.5  0.554  0.250   2.9  13.6   8.6  33.7   3.0   0.4   
2    72  72.0  2578.0  28.2  0.348  0.361   2.0  21.8  12.4  24.2   1.3   2.5   
3    54   4.0   508.0  27.7  0.000  0.655  16.9  26.2  21.7   6.6   1.2   3.5   
4    80  80.0  2750.0  27.6  0.236  0.397   6.1  18.1  12.4  49.6   2.9   0.6   
..   ..   ...     ...   ...    ...    ...   ...   ...   ...   ...   ...   ...   
372   8   3.0   144.0   1.8  0.262  0.569   1.5  14.9   7.9  23.3   1.0   0.0   
373   8   0.0    39.0   1.7  0.300  0.000   0.0   0.0   0.0  10.8   3.7   0.0   
374  12   0.0    42.0  -2.7  0.474  0.211   0.0  10.3   5.2   3.5   0.0   0.0   
375   5   0.0    27.0  -4.8  0.375  0.250   0.0   4.2   2.1   5.0   1.9   0.0   
376   6   1.0    43.0  -7.7  0.583  0.000   2.7   2.6   2.7   3.2   2.4   1.9   

     TOV%  USG%   ORtg   DRtg   OWS  DWS    WS  WS/48  OBPM  DBPM   BPM  VORP  
0     0.0  30.2  140.0  110.0   0.0  0.0   0.0  0.291   6.3 -10.5  -4.1   0.0  
1    12.9  32.6  125.0  103.0  13.8  4.1  17.9  0.318  12.4   0.1  12.5   9.8  
2    13.5  30.6  122.0  104.0  11.0  3.5  14.5  0.270   7.0   0.9   7.9   6.4  
3    11.5  22.8  130.0   96.0   2.3  1.2   3.4  0.325   2.7   0.9   3.6   0.7  
4    16.8  31.6  115.0  103.0  10.0  4.0  14.0  0.245   7.6   2.4  10.0   8.3  
..    ...   ...    ...    ...   ...  ...   ...    ...   ...   ...   ...   ...  
372  26.3  33.5   70.0  112.0  -0.8  0.0  -0.8 -0.263  -9.9  -3.5 -13.4  -0.4  
373  33.3  16.5   54.0  109.0  -0.2  0.0  -0.1 -0.168  -9.3  -2.7 -12.0  -0.1  
374  16.2  25.7   63.0  114.0  -0.2  0.0  -0.2 -0.238 -11.0  -5.9 -16.9  -0.2  
375  25.3  19.5   57.0  110.0  -0.1  0.0  -0.1 -0.199 -10.3  -2.7 -13.0  -0.1  
376  25.0  16.9   29.0  107.0  -0.3  0.0  -0.3 -0.283 -11.4  -0.2 -11.6  -0.1  

[376 rows x 24 columns]

In [25]:
X, y = perf_sal_df[perf_columns].to_numpy(), perf_sal_df['SALARY'].to_numpy()

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)

In [27]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [28]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Train R2 score: {r2_score(y_train, model.predict(X_train))}')
print(f'Test R2 score: {r2_score(y_test, y_pred)}')

Train R2 score: 0.5117947015859956
Test R2 score: 0.4780297299050488


### adding the weight, height and age

In [29]:
perf_sal_df = final_merge.loc[:, perf_columns + ['Ht', 'Wt', 'New age'] +['SALARY']]
perf_sal_df.dropna(inplace=True)
perf_sal_df.head()

G    GS      MP   PER   3PAr    FTr  ORB%  DRB%  TRB%  AST%  STL%  BLK%  \
0   2   0.0     6.0  32.9  0.250  0.000   0.0  17.9   9.1   0.0   0.0   0.0   
1  79  79.0  2700.0  31.5  0.554  0.250   2.9  13.6   8.6  33.7   3.0   0.4   
2  72  72.0  2578.0  28.2  0.348  0.361   2.0  21.8  12.4  24.2   1.3   2.5   
3  54   4.0   508.0  27.7  0.000  0.655  16.9  26.2  21.7   6.6   1.2   3.5   
4  80  80.0  2750.0  27.6  0.236  0.397   6.1  18.1  12.4  49.6   2.9   0.6   

   TOV%  USG%   ORtg   DRtg   OWS  DWS    WS  WS/48  OBPM  DBPM   BPM  VORP  \
0   0.0  30.2  140.0  110.0   0.0  0.0   0.0  0.291   6.3 -10.5  -4.1   0.0   
1  12.9  32.6  125.0  103.0  13.8  4.1  17.9  0.318  12.4   0.1  12.5   9.8   
2  13.5  30.6  122.0  104.0  11.0  3.5  14.5  0.270   7.0   0.9   7.9   6.4   
3  11.5  22.8  130.0   96.0   2.3  1.2   3.4  0.325   2.7   0.9   3.6   0.7   
4  16.8  31.6  115.0  103.0  10.0  4.0  14.0  0.245   7.6   2.4  10.0   8.3   

       Ht   Wt  New age    SALARY  
0  200.66  205       23     30888  
1  190.50  185       27  11370786  
2  205.74  240       27  20158622  
3  220.98  290       27   1200000  
4  190.50  200       27  16744218

In [30]:
X, y = perf_sal_df[perf_columns + ['Ht', 'Wt', 'New age'] ].to_numpy(), perf_sal_df['SALARY'].to_numpy()

In [31]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Train R2 score: {r2_score(y_train, model.predict(X_train))}')
print(f'Test R2 score: {r2_score(y_test, y_pred)}')

Train R2 score: 0.5487321626209161
Test R2 score: 0.557791514802406


## Adding the first categorical variable: The team

In [32]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline

### Ordinal encoding

In [33]:
cat_pipeline = Pipeline([
    ("ordinal_encoder", OrdinalEncoder())
])

categorical_preprocessing = ColumnTransformer([
    ("categorical_preproc", cat_pipeline, ['Tm'])
])

In [34]:
final_merge['oe'] = categorical_preprocessing.fit_transform(final_merge)

In [35]:
df = final_merge[perf_columns + ['Ht', 'Wt', 'New age'] + ['oe']+ ['SALARY']]
df.dropna(inplace=True)

In [36]:
X, y = df[perf_columns + ['Ht', 'Wt', 'New age'] + ['oe']].to_numpy(), df['SALARY'].to_numpy()

In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Train R2 score: {r2_score(y_train, model.predict(X_train))}')
print(f'Test R2 score: {r2_score(y_test, y_pred)}')

Train R2 score: 0.550867588784258
Test R2 score: 0.5610492688323184


## One-hot encoding (team column)

In [38]:
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([
    ("ohe", OneHotEncoder())
])

categorical_preprocessing = ColumnTransformer([
    ("categorical_preproc", cat_pipeline, ['Tm'])
])

In [39]:
ohe_array = categorical_preprocessing.fit_transform(final_merge).todense()
ohe_array.shape

(377, 31)

In [40]:
final_merge.loc[:,[str(i) for i in range(ohe_array.shape[1])]] = ohe_array

In [41]:
df = final_merge[perf_columns + ['Ht', 'Wt', 'New age'] +[str(i) for i in range(ohe_array.shape[1])]+ ['SALARY']]
df.dropna(inplace=True)

In [42]:
X, y = df[perf_columns + ['Ht', 'Wt', 'New age'] ].to_numpy(), df['SALARY'].to_numpy()

In [43]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Train R2 score: {r2_score(y_train, model.predict(X_train))}')
print(f'Test R2 score: {r2_score(y_test, y_pred)}')

Train R2 score: 0.5487321626209161
Test R2 score: 0.557791514802406


## Trying the target encoding for the team column

In [44]:
df = final_merge[perf_columns + ['Ht', 'Wt', 'New age', 'Tm'] + ['SALARY']]
df.dropna(inplace=True)

In [45]:
X, y = df[perf_columns + ['Ht', 'Wt', 'New age', 'Tm'] ], df['SALARY']

In [46]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)

In [47]:
from category_encoders import TargetEncoder

In [48]:
TargetEncoder?

In [49]:
encoder = TargetEncoder()
X_train['Team_encoded'] = encoder.fit_transform(X_train['Tm'], y_train)

In [50]:
X_train.head()

G    GS      MP   PER   3PAr    FTr  ORB%  DRB%  TRB%  AST%  STL%  BLK%  \
278  53   0.0   496.0  11.1  0.008  0.403   8.5  22.8  15.8   9.8   1.6   1.6   
60   25  25.0   765.0  17.9  0.271  0.296   2.8  16.4   9.3  34.6   2.1   0.9   
119  47   4.0  1070.0  15.7  0.159  0.145   5.7   9.3   7.5   7.2   1.7   1.2   
266  17   1.0   176.0  11.3  0.305  0.203   3.1  10.4   6.8   8.3   2.5   0.5   
353  64  33.0  1301.0   6.4  0.457  0.097   1.6  14.3   8.1   6.9   0.6   1.0   

     TOV%  USG%   ORtg   DRtg  OWS  DWS   WS  WS/48  OBPM  DBPM  BPM  VORP  \
278  13.1  15.0  103.0  102.0  0.3  0.8  1.0  0.101  -3.8   1.4 -2.4   0.0   
60   17.1  24.7  105.0  109.0  0.8  0.6  1.4  0.087   1.7   0.0  1.7   0.7   
119   6.4  19.9  112.0  110.0  1.7  0.6  2.2  0.101   0.2  -1.3 -1.2   0.2   
266   4.5  16.2  106.0  109.0  0.1  0.1  0.3  0.071  -1.5  -1.1 -2.6   0.0   
353  12.8  13.7   93.0  109.0 -0.5  1.0  0.4  0.016  -3.4  -0.7 -4.1  -0.7   

         Ht   Wt  New age   Tm  Team_encoded  
278  208.28  273       33  TOT  2.836529e+06  
60   198.12  220       26  NOP  4.773246e+06  
119  203.20  230       22  PHO  3.562605e+06  
266  200.66  228       35  SAC  6.811479e+06  
353  200.66  200       24  CHI  7.457172e+06

In [51]:
X_test['Team_encoded'] = encoder.transform(X_test['Tm'])

In [52]:
X_train, y_train = X_train[perf_columns + ['Ht', 'Wt', 'New age', 'Team_encoded'] ].to_numpy(), y_train.to_numpy()

In [53]:
X_test, y_test = X_test[perf_columns + ['Ht', 'Wt', 'New age', 'Team_encoded'] ].to_numpy(), y_test.to_numpy()

In [54]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Train R2 score: {r2_score(y_train, model.predict(X_train))}')
print(f'Test R2 score: {r2_score(y_test, y_pred)}')

Train R2 score: 0.5753786773065073
Test R2 score: 0.5384670254640933


In [55]:
from sklearn.preprocessing import StandardScaler

In [56]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_x.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1,1))

In [57]:
X_test_scaled = scaler_x.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.reshape(-1,1))

In [58]:
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)
y_pred = model.predict(X_test_scaled)
print(f'Train R2 score: {r2_score(y_train_scaled, model.predict(X_train_scaled))}')
print(f'Test R2 score: {r2_score(y_test_scaled, y_pred)}')

Train R2 score: 0.5753786773065074
Test R2 score: 0.5384670254642678


# Adding the second categorical variable: Race

In [59]:
cat_pipeline = Pipeline([
    ("ordinal_encoder", OrdinalEncoder())
])

categorical_preprocessing = ColumnTransformer([
    ("categorical_preproc", cat_pipeline, ['Tm', 'Race'])
])

In [60]:
final_merge[['team_oe', 'race_oe']] = categorical_preprocessing.fit_transform(final_merge)

In [61]:
df = final_merge[perf_columns + ['Ht', 'Wt', 'New age'] + ['team_oe', 'race_oe']+ ['SALARY']]
df.dropna(inplace=True)

In [62]:
X, y = df[perf_columns + ['Ht', 'Wt', 'New age'] +  ['team_oe', 'race_oe']].to_numpy(), df['SALARY'].to_numpy()

In [63]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Train R2 score: {r2_score(y_train, model.predict(X_train))}')
print(f'Test R2 score: {r2_score(y_test, y_pred)}')

Train R2 score: 0.5527370497347361
Test R2 score: 0.5654957273937653
